# GPT Model

In this notebook we will implement the GPT-2 model. 

To start, we implement at dummy class that shows the architecture of the neural network. 

Then, we implement the layers we mocked in the dummy GPT model.

Finally, we put everything back together with a few additions.

In [26]:
import torch
import torch.nn as nn

## Dummy GPT

We implement a dummy GPT class below, with the following architecture:


Architecture:
```
DummyGPT(
  (dropout): Dropout(p=0.1, inplace=False)
  (tokenEmbed): Embedding(50257, 768)
  (posEmbed): Embedding(1024, 768)
  (normLayer): DummyLayerNorm()
  (trfBlocks): Sequential(
    (0): DummyTransformer()
    (1): DummyTransformer()
    (2): DummyTransformer()
    (3): DummyTransformer()
    (4): DummyTransformer()
    (5): DummyTransformer()
    (6): DummyTransformer()
    (7): DummyTransformer()
    (8): DummyTransformer()
    (9): DummyTransformer()
    (10): DummyTransformer()
    (11): DummyTransformer()
  )
  (outLayer): Linear(in_features=768, out_features=50257, bias=False)
)
```

The model will be initialized using a configuration dictionary that will contain the number of layers and other model parameters. 

We use the parameters from GPT-2 for reference:

In [27]:
cfg = {
    "vocabSize": 50257, # Number of tokens in vocabulary.
    "contextLength": 256, # Max. number of tokens the LLM sees per run.
    "embDim": 768, # Size of the internal. embeddings used in the LLM attention mechanism.
    "nLayers" : 12, # Number of transformer layers.
    "dropRate": 0.1, # Feature dropout rate.
    "nHeads": 12 # Num. Attention heads per multi-head attention block
    
}


In [28]:
class DummyLayerNorm(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x

class DummyTransformer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x

In [29]:
class DummyGPT(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # Dropout
        self.dropout = nn.Dropout(cfg["dropRate"])
        # Token embedding
        self.tokenEmbed = nn.Embedding(cfg["vocabSize"], cfg["embDim"])
        # Positional embedding
        self.posEmbed = nn.Embedding(cfg["contextLength"], cfg["embDim"])
        # Normalization
        self.normLayer = DummyLayerNorm()
        # Transformer Blocks
        self.trfBlocks = nn.Sequential(*[
            DummyTransformer() for _ in range(cfg["nLayers"])
        ])
        # Linear projection.
        self.outLayer = nn.Linear(cfg["embDim"], cfg["vocabSize"], bias=False)

    def forward(self, x):
        b, seqLen = x.shape
        tokEmbeds = self.tokenEmbed(x)
        posEmbeds = self.posEmbed(torch.arange(seqLen, device=x.device))
        x = tokEmbeds + posEmbeds
        x = self.dropout(x)
        x = self.normLayer(x)
        for l in self.trfBlocks:
            x = l(x)
        return self.outLayer(x)

In [30]:
# Initialize model.
dummyGpt = DummyGPT(cfg)
dummyGpt

DummyGPT(
  (dropout): Dropout(p=0.1, inplace=False)
  (tokenEmbed): Embedding(50257, 768)
  (posEmbed): Embedding(256, 768)
  (normLayer): DummyLayerNorm()
  (trfBlocks): Sequential(
    (0): DummyTransformer()
    (1): DummyTransformer()
    (2): DummyTransformer()
    (3): DummyTransformer()
    (4): DummyTransformer()
    (5): DummyTransformer()
    (6): DummyTransformer()
    (7): DummyTransformer()
    (8): DummyTransformer()
    (9): DummyTransformer()
    (10): DummyTransformer()
    (11): DummyTransformer()
  )
  (outLayer): Linear(in_features=768, out_features=50257, bias=False)
)

In [31]:
# Define a batch to be used for testing purposes
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)

In [32]:
# Initialize model and run model on batch.
# Model isn't trained and all ouputs are gibberish.
torch.manual_seed(123)

logits = dummyGpt(batch)  # shape: [batch, seqLen, vocabSize]
pred_ids = logits.argmax(dim=-1)  # shape: [batch, seqLen]
print(batch)
for i in range(pred_ids.shape[0]):
    print(tokenizer.decode(pred_ids[i].tolist()))


tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])
705 striving benz Optional
705brothererno patriarchal


## Layer Normalization

Next step is to implement the layer normalization class.

Layer normalization is used to keep a stable model training, by reducing internal co-variate shift.  

Let's explain in more details:

- Covariate: Refers to the input features of a ML model.
- Shift : Distribution change.
- Internal : Refers to the internal covariates, which are the internal activations of the neural networks.


### Why does layer normalization help reduce internal covariate shift ?

While the model is trained, model weights are updated at each training step. Without normalization, each layer is forced to learn the new distribution of the previous layer outputs. Layer normalization helps by making the mean = 0 and variance = 1, maintaining the input feature distribution regardless of the iteration.

### How does it work ?

For an input vector `X = [x1, x2, ..., xn]`, layer normalization works by computing:

`z =  (  xi  - mean(X)  )  /  std(X) `

In practice, layer normalization includes two learnable terms that allow the model to rescale and shift the inputs, learning better representations of the input activations:

`z = γ * (  xi  - mean(X)  )  /  ( std(X) + ε )    +    β `

Where:
- `γ`: Scale
- `ε`: Small Epsilon, avoid division by 0.
- `β`: Shift

In [33]:
class LayerNorm(nn.Module):
    def __init__(self, embDim):
        super().__init__()
        # Scale parameter.
        self.scale = nn.Parameter(torch.ones(embDim))
        # Shift parameter.
        self.shift = nn.Parameter(torch.zeros(embDim))
        # Epsilon constant. Avoids division by zero.
        self.eps = 1e-8
    
    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var  = x.var(dim=-1, keepdim=True, unbiased=False) 
        normX = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * normX + self.shift

In [34]:
# Try LayerNorm.
torch.manual_seed(123)
torch.set_printoptions(sci_mode=False)

batch = torch.randn(2, 5)
print("Mean:\n ", batch.mean(dim=-1))
print("Variance :\n ", batch.var(dim=-1,  unbiased=False))

ln = LayerNorm(5)
out = ln(batch)
print("Norm. Mean:\n ", out.mean(dim=-1))
print("Norm. Variance :\n ", out.var(dim=-1,  unbiased=False))

Mean:
  tensor([-0.3596, -0.2606])
Variance :
  tensor([0.2015, 0.2673])
Norm. Mean:
  tensor([    -0.0000,      0.0000], grad_fn=<MeanBackward1>)
Norm. Variance :
  tensor([1.0000, 1.0000], grad_fn=<VarBackward0>)


## GELU activation

In [35]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2/torch.pi)) * 
            (x + 0.044715 * torch.pow(x,3))
        ))

In [36]:
GELU()(batch)

tensor([[-0.0508,  0.0659, -0.1315, -0.0974, -0.1387],
        [ 0.1220, -0.1610, -0.1700,  0.2031, -0.0496]])

## FeedForward

In [37]:
class FeedForwardModule(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        embDim = cfg["embDim"]

        # Sequence of layers, linear -> GELU -> linear. 
        # Note that the first linear layer expands the dimension of the input,
        # whereas the second one, reduces the dimension to the original embed-dimension size.
        self.layers = nn.Sequential(
            nn.Linear(embDim, 4 * embDim),
            GELU(),
            nn.Linear(4 * embDim, embDim)
        )

    def forward(self, x):
        return self.layers(x)

In [38]:
FeedForwardModule({"embDim": 5})(batch)

tensor([[-0.0692,  0.1527, -0.0808,  0.0560,  0.1473],
        [ 0.0020,  0.1303, -0.2325,  0.0055,  0.2685]],
       grad_fn=<AddmmBackward0>)

## Transformer Module

In [39]:
# Load attention modules from previous ipynb.
%run coding-attention-mechanisms.ipynb

tensor([[0.2685, 0.7413],
        [0.2738, 0.7564],
        [0.2668, 0.7366],
        [0.2618, 0.7218],
        [0.2712, 0.7495]], grad_fn=<MmBackward0>)
tensor([[-0.4927, -0.0791],
        [-0.4938, -0.0806],
        [-0.4924, -0.0851],
        [-0.4923, -0.0819],
        [-0.4928, -0.0853]], grad_fn=<MmBackward0>)
Self Attention V2 output: 
 tensor([[-0.4927, -0.0791],
        [-0.4938, -0.0806],
        [-0.4924, -0.0851],
        [-0.4923, -0.0819],
        [-0.4928, -0.0853]], grad_fn=<MmBackward0>)
Self Attention V1 output: 
 tensor([[-0.4927, -0.0791],
        [-0.4938, -0.0806],
        [-0.4924, -0.0851],
        [-0.4923, -0.0819],
        [-0.4928, -0.0853]], grad_fn=<MmBackward0>)
batch.shape: torch.Size([2, 5, 3])
contextVecs.shape: torch.Size([2, 5, 2])
batch.shape: torch.Size([2, 5, 3])
contextVecs.shape: torch.Size([2, 5, 4])
batch.shape: torch.Size([2, 5, 3])
contextVecs.shape: torch.Size([2, 5, 2])
batch.shape: torch.Size([2, 1024, 768])
contextVecs.shape: torch.Size(

In [40]:
class TransformerBlock(nn.Module):
    
    def __init__(self, cfg):
        super().__init__()
        embDim = cfg["embDim"]
        self.layerNorm1 = LayerNorm(embDim)
        self.layerNorm2 = LayerNorm(embDim)
        self.dropout = nn.Dropout(cfg["dropRate"])
        self.feedfwd = FeedForwardModule(cfg)
        self.attention = MultiHeadAttention(
            embDim, embDim, 
            contextLength=cfg["contextLength"],
            numHeads=cfg["nHeads"],
            dropout=cfg["dropRate"]
        )

        
    def forward(self, x):
        shortcut = x

        # LayerNorm
        x = self.layerNorm1(x)
        # MultiHeadAttention
        x = self.attention(x)
        # Dropout
        x = self.dropout(x)
        # Shortcut
        x = x + shortcut

        shortcut = x
        # LayerNorm
        x = self.layerNorm2(x)
        # Forward
        x = self.feedfwd(x)
        # Dropout
        x = self.dropout(x)
        # Shortcut
        x = x + shortcut

        return x

In [41]:
# Test transformer block
TransformerBlock(cfg)(torch.ones(1,1,768))[:,:,:10]

tensor([[[0.9744, 0.7451, 1.1016, 1.0350, 1.0850, 1.0020, 0.8070, 0.7283,
          1.0585, 1.0846]]], grad_fn=<SliceBackward0>)

# GPT Module

In [42]:
class GPTModel(nn.Module):

    def __init__(self, cfg):
        super().__init__()
        self.tokEmbed = nn.Embedding(cfg["vocabSize"], cfg["embDim"])
        self.posEmbed = nn.Embedding(cfg["contextLength"], cfg["embDim"])
        self.trfBlocks = nn.Sequential(*[
            TransformerBlock(cfg) for _ in range(cfg["nLayers"])
        ])
        self.dropout = nn.Dropout(cfg["dropRate"])
        self.finalNorm = LayerNorm(cfg["embDim"])
        self.outLayer = nn.Linear(cfg["embDim"], cfg["vocabSize"])

    def forward(self, x):
        # TokEmbed & PosEmbed
        b, seqLen = x.shape
        tokens = self.tokEmbed(x)
        pos = self.posEmbed(torch.arange(seqLen, device=x.device))
        x = tokens + pos
        # Dropout
        x = self.dropout(x)
        # Transformer[]
        x = self.trfBlocks(x)
        # LayerNorm
        x = self.finalNorm(x)
        # Output heads
        logits = self.outLayer(x)
        
        return logits

In [43]:
# Test the GPT Moduel
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)

model = GPTModel(cfg)

### Generating Text

- **ArgMax**: Choose the token with the highest score.

- **TopK**: Choose only from the topK token scores.

- **Temperature**: Logits are divided by temperature. A temperature > 1 scales values, making them more likely. A temperature < 1 makes tokens with highest score more likely to be selected

In [56]:
# Define a function that runs the model `maxNewTokens` times.
# It appends the predicted text to the input `idx` text. 
# The output is encoded, it needs to be decoded with the tokenizer.
def generateText(model: nn.Module, idx: tuple, maxNewTokens: int, contextSize: int, 
                 temperature : int = 0.0, top_k=None, eos_id=None):
    for i in range(maxNewTokens):
        idxCond = idx[:, -contextSize:]

        with torch.no_grad():
            logits = model(idxCond)
        logits = logits[:, -1, :]
               
        # Apply topk if specified.
        if top_k:
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(logits < min_val, 
                                 torch.tensor(float('-inf')).to(logits.device),
                                 logits)

        if temperature > 0.0:
            logits = logits / temperature
            probas = torch.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probas, num_samples=1)
        else:
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)
        if idx_next == eos_id:
            break
        idx = torch.cat((idx, idx_next), dim=1)
    return idx

In [57]:
# Generate start tensor.
startContext = "Hello, I am"
encoded = tokenizer.encode(startContext)
print("encoded:", encoded)
encodedTensor = torch.tensor(encoded).unsqueeze(0)
print("encoded_tensor.shape:", encodedTensor.shape)

encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


In [58]:
# Run GPT and decode outputs.
model.eval()
out = generateText(
    model=model,
    idx=encodedTensor,
    maxNewTokens=6,
    contextSize=cfg["contextLength"]
)

print("Output:", out)
print("Output length:", len(out[0]))
decodedText = tokenizer.decode(out.squeeze(0).tolist())
print(decodedText)

Output: tensor([[15496,    11,   314,   716, 24111, 43446, 12663, 18650, 28505, 27960]])
Output length: 10
Hello, I amFrench rebirth fundra Oracle Worm Midnight
